# CHAPTER1: Common data problems

## Ride Sharing Data Set
#### Encoding
"user_type" -> 1 for free riders., 2 for pay per ride., 3 for monthly subscribers.

Exposure to some common problems that you may encounter with your data,
- data type constraints,
- data range constrains,
- uniqueness constraints,
- membership constraints for categorical values.

In [59]:
import pandas as pd
import numpy as np

# CHAPTER1: Common data problems

## To Be Described()

In [2]:
ride_sharing_df = pd.read_csv("./data/cleaningData/ride_sharing_new.csv")
ride_sharing_df.head()

,Unnamed: 0,duration,station_A_id,station_A_name,station_B_id,station_B_name,bike_id,user_type,user_birth_year,user_gender
0,0,12 minutes,81,Berry St at 4th St,323,Broadway at Kearny,5480,2,1959,Male
1,1,24 minutes,3,Powell St BART Station (Market St at 4th St),118,Eureka Valley Recreation Center,5193,2,1965,Male
2,2,8 minutes,67,San Francisco Caltrain Station 2 (Townsend St...,23,The Embarcadero at Steuart St,3652,3,1993,Male
3,3,4 minutes,16,Steuart St at Market St,28,The Embarcadero at Bryant St,1883,1,1979,Male
4,4,11 minutes,22,Howard St at Beale St,350,8th St at Brannan St,4626,2,1994,Male


In [3]:
ride_sharing_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25760 entries, 0 to 25759
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0       25760 non-null  int64 
 1   duration         25760 non-null  object
 2   station_A_id     25760 non-null  int64 
 3   station_A_name   25760 non-null  object
 4   station_B_id     25760 non-null  int64 
 5   station_B_name   25760 non-null  object
 6   bike_id          25760 non-null  int64 
 7   user_type        25760 non-null  int64 
 8   user_birth_year  25760 non-null  int64 
 9   user_gender      25760 non-null  object
dtypes: int64(6), object(4)
memory usage: 2.0+ MB


In [4]:
print(ride_sharing_df['user_type'].describe())

count    25760.000000
mean         2.008385
std          0.704541
min          1.000000
25%          2.000000
50%          2.000000
75%          3.000000
max          3.000000
Name: user_type, dtype: float64


#### user_type is categorical Data Type  ...  mean, std, median has no mean here!
<br>
<br>
<br>
<br>


In [5]:
ride_sharing_df['user_type_cat'] = ride_sharing_df['user_type'].astype('category')

In [6]:
ride_sharing_df['user_type_cat'].describe()

count     25760
unique        3
top           2
freq      12972
Name: user_type_cat, dtype: int64

In [7]:
assert ride_sharing_df['user_type_cat'].dtype == 'category'  ## confirming the change

In [12]:
ride_sharing_df['duration_trim'] = ride_sharing_df['duration'].str.strip('minutes')

In [13]:
ride_sharing_df['duration_time'] = ride_sharing_df['duration_trim'].astype('int')

In [14]:
assert ride_sharing_df['duration_time'].dtype == 'int'

In [15]:
ride_sharing_df['duration_time'].describe()

count    25760.000000
mean        11.389053
std         23.500516
min          1.000000
25%          6.000000
50%          9.000000
75%         13.000000
max       1372.000000
Name: duration_time, dtype: float64

# DATASET Missed 'tire_sizes' 

Bicycle tire sizes could be either 26″, 27″ or 29″ and are here correctly stored as a categorical value. In an effort to cut maintenance costs, the ride sharing provider decided to set the maximum tire size to be 27″.

In [ ]:
ride_sharing_df['tire_sizes'].dtype
ride_sharing['tire_sizes'] = ride_sharing['tire_sizes'].astype('int')
ride_sharing.loc[ride_sharing['tire_sizes'] > 27, 'tire_sizes'] = 27
ride_sharing['tire_sizes'] = ride_sharing['tire_sizes'].astype('category')
ride_sharing['tire_sizes'].describe()

# DATASET Missed 'ride_date' 

## Identifing Complete&Incomplete 
## Treating Duplication

In [ ]:
ride_sharing['ride_dt'] = pd.to_datetime(ride_sharing['ride_date']).dt.date
today = dt.date.today()
ride_sharing.loc[ride_sharing['ride_dt'] > today, 'ride_dt'] = today
ride_sharing['ride_dt'].max()

using the default .duplicated() without tweaking some of the arguments returns only the non-first complete duplicates across all columns.



Correct! Subsetting on metadata and keeping all duplicate records gives you a better bird-eye's view over your data and how to duplicate it! You can even subset the loans DataFrame using bracketing and sort the values so you can properly identify the duplicates.



In [20]:
# nothing is duplicated accross all attributes
_filter_ = ride_sharing_df.duplicated()
ride_sharing_df[_filter_]

,Unnamed: 0,duration,station_A_id,station_A_name,station_B_id,station_B_name,bike_id,user_type,user_birth_year,user_gender,user_type_cat,duration_trim,duration_time


# DATASET Missed 'ride_id'

In [ ]:
duplicates = ride_sharing.duplicated(subset = 'ride_id', keep = False)
duplicated_rides = ride_sharing[duplicates].sort_values('ride_id')
print(duplicated_rides[['ride_id','duration','user_birth_year']])

new update feeding into ride_sharing contains a bug generating both complete and incomplete duplicated rows for some values of the ride_id column, with occasional discrepant values for the user_birth_year and duration columns.

## Treating duplicaates

In [ ]:
# Drop complete duplicates from ride_sharing; and we know it is not exist
ride_dup = ride_sharing.drop_duplicates()

In [ ]:
# Create statistics dictionary for aggregation function
statistics = {'user_birth_year': 'min', 'duration': 'mean'}

# Group by ride_id and compute new statistics
ride_unique = ride_dup.groupby('ride_id').agg(statistics).reset_index()

In [ ]:
# Find duplicated values again
duplicates = ride_unique.duplicated(subset = 'ride_id', keep = False)
duplicated_rides = ride_unique[duplicates] # should not see nothing

In [ ]:
assert duplicated_rides.shape[0] == 0

# CHAPTER2: Text and categorical data problems

Categorical and text data can often be some of the messiest parts of a dataset due to their unstructured nature. In this chapter, you’ll learn how to fix whitespace and capitalization inconsistencies in category labels, collapse multiple categories into one, and reformat strings for consistency.


### membership constraints
- A day_of_the_week has value saturwednesday
- A month Columns has value 14
- A has_loan with value 12
- A gpa with value -Z

In [43]:
categories = pd.DataFrame({
    'cleanliness': ['Clean', 'Average', 'Somewhat clean', 'Somewhat dirty', 'Dirty'],
    'safety': ['Neutral', 'Very safe', 'Somewhat safe', 'Very unsafe', 'Somewhat unsafe'],
    'satisfaction': ['Very satisfied', 'Neutral', 'Somewhat satisfied', 'Somewhat unsatisfied', 'Very unsatisfied']
})
categories

,cleanliness,safety,satisfaction
0,Clean,Neutral,Very satisfied
1,Average,Very safe,Neutral
2,Somewhat clean,Somewhat safe,Somewhat satisfied
3,Somewhat dirty,Very unsafe,Somewhat unsatisfied
4,Dirty,Somewhat unsafe,Very unsatisfied


In [41]:
airlines = pd.read_csv("./data/cleaningData/airlines_final.csv")
airlines.head()

,Unnamed: 0,id,day,airline,destination,dest_region,dest_size,boarding_area,dept_time,wait_min,cleanliness,safety,satisfaction
0,0,1351,Tuesday,UNITED INTL,KANSAI,Asia,Hub,Gates 91-102,2018-12-31,115.0,Clean,Neutral,Very satisfied
1,1,373,Friday,ALASKA,SAN JOSE DEL CABO,Canada/Mexico,Small,Gates 50-59,2018-12-31,135.0,Clean,Very safe,Very satisfied
2,2,2820,Thursday,DELTA,LOS ANGELES,West US,Hub,Gates 40-48,2018-12-31,70.0,Average,Somewhat safe,Neutral
3,3,1157,Tuesday,SOUTHWEST,LOS ANGELES,West US,Hub,Gates 20-39,2018-12-31,190.0,Clean,Very safe,Somewhat satsified
4,4,2992,Wednesday,AMERICAN,MIAMI,East US,Hub,Gates 50-59,2018-12-31,559.0,Somewhat clean,Very safe,Somewhat satsified


In [45]:
airlines['cleanliness'].unique()

array(['Clean', 'Average', 'Somewhat clean', 'Somewhat dirty', 'Dirty'],
      dtype=object)

In [46]:
airlines['safety'].unique()

array(['Neutral', 'Very safe', 'Somewhat safe', 'Very unsafe',
       'Somewhat unsafe'], dtype=object)

In [47]:
airlines['satisfaction'].unique()

array(['Very satisfied', 'Neutral', 'Somewhat satsified',
       'Somewhat unsatisfied', 'Very unsatisfied'], dtype=object)

In [49]:
# Find the cleanliness category in airlines not in categories
cat_clean = set(airlines['cleanliness']).difference(categories['cleanliness'])
cat_clean

set()

In [50]:
# Find rows with that category
cat_clean_rows = airlines['cleanliness'].isin(cat_clean)
airlines[cat_clean_rows]

,Unnamed: 0,id,day,airline,destination,dest_region,dest_size,boarding_area,dept_time,wait_min,cleanliness,safety,satisfaction


In [51]:
airlines = airlines[~cat_clean_rows]

## common problems affecting categorical variables in your data, 
including white spaces and inconsistencies in your categories,
and the problem of creating new categories
and mapping existing ones to new ones.

In [52]:
air_lines['dest_region'].unique()       # you need to Identify Inconsistency yourself

array(['Asia', 'Canada/Mexico', 'West US', 'East US', 'Midwest US',
       'EAST US', 'Middle East', 'Europe', 'eur', 'Central/South America',
       'Australia/New Zealand', 'middle east'], dtype=object)

In [54]:
air_lines['dest_size'].unique()

array(['Hub', 'Small', '    Hub', 'Medium', 'Large', 'Hub     ',
       '    Small', 'Medium     ', '    Medium', 'Small     ',
       '    Large', 'Large     '], dtype=object)

In [55]:
airlines['dest_region'] = airlines['dest_region'].str.lower() 
airlines['dest_region'] = airlines['dest_region'].replace({'eur':'europe'})

In [56]:
airlines['dest_size'] = airlines['dest_size'].str.strip()

In [57]:
print(airlines['dest_region'].unique())
print(airlines['dest_size'].unique())

['asia' 'canada/mexico' 'west us' 'east us' 'midwest us' 'middle east'
 'europe' 'central/south america' 'australia/new zealand']
['Hub' 'Small' 'Medium' 'Large']


<br>
<br>
<br>
<br>

The airlines DataFrame contains the day and wait_min columns, which are categorical and numerical respectively.<br> 
The day column contains the exact day a flight took place,<br>
and wait_min contains the amount of minutes it took travelers to wait at the gate. <br>
To make your analysis easier, you want to create two new categorical variables:

###### wait_type: 'short' for 0-60 min, 'medium' for 60-180 and long for 180+
###### day_week: 'weekday' if day is in the weekday, 'weekend' if day is in the weekend.

In [60]:
# Create ranges for categories
label_ranges = [0, 60, 180, np.inf]       # Bins for The Distribution  n value points -> (n-1) bins
label_names = ['short', 'medium', 'long'] # Labels for The Bins 

In [61]:
airlines['wait_type'] = pd.cut(airlines['wait_min'], bins = label_ranges, 
                               labels = label_names)

In [62]:
# Create mappings and replace
mappings = {'Monday':'weekday', 'Tuesday':'weekday', 'Wednesday': 'weekday', 
            'Thursday': 'weekday', 'Friday': 'weekday', 
            'Saturday': 'weekend', 'Sunday': 'weekend'}

In [63]:
airlines['day_week'] = airlines['day'].replace(mappings)

# airlines Missed 'full_name'

Mr. Ahmed Khaled <br>
Eng. Mahdi Abied <br>
.. 
## CHALLENGE: Is There is any regex Expression captures honorifics pattern ? 
in data set Miss does not have a perion (.) like others makes it harder
##### Your ultimate objective is to create two new columns named first_name and last_name, containing the first and last names of respondents respectively. Before doing so however, you need to remove honorifics.

In [ ]:
# airlines['full_name'] = airlines['full_name'].str.replace("Dr.", "")

# airlines['full_name'] = airlines['full_name'].str.replace("Mr.", "")

# airlines['full_name'] = airlines['full_name'].str.replace("Miss", "")

# airlines['full_name'] = airlines['full_name'].str.replace("Ms.", "")

In [ ]:
# assert airlines['full_name'].str.contains('Ms.|Mr.|Miss|Dr.').any() == False

# airlines Missed 'survey_response'
That was awufull. <br>
The food were too expensive <br>
...

In [ ]:
resp_length = airlines['survey_response'].str.len()
resp_length                                        # boolean Series

In [ ]:
airlines_survey = airlines[ resp_length > 40 ]
airlines_survey

In [ ]:
assert airlines_survey['survey_response'].str.len().min() > 40

In [ ]:
airlines_survey['survey_response']

Phenomenal work! These types of feedbacks are essential to improving any service. Coupled with some wordcount analysis, you can find common patterns across all survey responses in no time!

# CHAPTER3: Advanced data problems

advanced data cleaning problems, such as ensuring that weights are all written in kilograms instead of pounds. You’ll also gain invaluable skills that will help you verify that values have been added correctly and that missing values don’t negatively impact your analyses.